In [2]:
import sdv
print(sdv.__version__)

1.14.0


In [3]:
# from data_synthesizer.util import  plot_training_loss, ModelType
from data_loader import DataLoader
from data_synthesizer.sdv import SDVCTGAN_
from data_evaluator import histo_plot_utility_compare, ClassifierType, MultivariateEvaluator, UtilityEvaluation, accuracy_compare
from data_evaluator.privacy_evaluation import SimilarityType
# from evaluation_report import EvaluationReportBuilder, EvaluationType, EvaluationReport
from data_synthesizer.pipeline.evaluation_task import ResemblanceEvaluationTask

### ctgan credit

In [2]:
cat_list_credit_card = ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6', 'default.payment.next.month']
# cat_list_credit_card = ['EDUCATION', 'MARRIAGE', 'PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6', 'default.payment.next.month']
num_list_credit_card = ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4','BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3','PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
df_real_credit_card_train = DataLoader('../data/credit_card_Train.csv').get_dataframe(cat_list_credit_card, str, drop_identation=True)
df_real_credit_card_test = DataLoader('../data/credit_card_Test.csv').get_dataframe(cat_list_credit_card, str, drop_identation=True)
df_real_credit_card_ctgan_holdout = DataLoader('../data/credit_card_Test.csv').get_dataframe(cat_list_credit_card, drop_identation=True)

In [ ]:
cat_list_adult = ['workclass','education','marital-status','occupation','relationship','race','sex','native-country','income']

In [3]:
from sdv.metadata import SingleTableMetadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df_real_credit_card_train)
ctgan = SDVCTGAN_(metadata, df_real_credit_card_train, verbose=False, cuda=True, epochs=1500)

discrete_columns :  ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'default.payment.next.month']


KeyboardInterrupt: 

In [5]:
ctgan.fit(df_real_credit_card_train)

_fit from child
got it


In [6]:
data = ctgan.sample(24000)

In [7]:
data.to_csv('../data/credit_card_ctgan_test_new.csv', sep=',', index=False)

In [15]:
df_real_credit_card_train = DataLoader('../data/credit_card_Train.csv').get_dataframe(cat_list_credit_card, drop_identation=True)
df_real_credit_card_ctgan_test_new = DataLoader('../data/credit_card_ctgan_test_new.csv').get_dataframe(cat_list_credit_card)
df_real_credit_card_ctgan_third = DataLoader('../data/credit_card_ctgan_training_third.csv').get_dataframe(cat_list_credit_card)


In [7]:
df_real_credit_card_ctgan_test_new

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,10000.0,1,2,2,22,0,0,0,0,0,...,24966.0,23149.0,12198.0,1280.0,2016.0,590.0,636.0,437.0,524.0,0
1,185395.0,2,2,2,23,-1,-1,-1,-1,-1,...,-959.0,-1228.0,6834.0,6988.0,3923.0,1553.0,13647.0,11223.0,3346.0,1
2,13319.0,1,1,2,23,0,0,0,0,0,...,10525.0,6977.0,28702.0,4868.0,1332.0,1808.0,0.0,385.0,3036.0,0
3,264248.0,2,2,2,24,0,0,-1,-1,-2,...,2084.0,-1906.0,2440.0,1800.0,8748.0,589.0,19.0,0.0,259.0,1
4,355653.0,1,2,1,42,-1,-1,-1,-1,-1,...,3197.0,6782.0,-396.0,1087.0,7963.0,1222.0,8280.0,404.0,9.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,23508.0,1,2,2,23,2,0,0,0,0,...,34617.0,40897.0,35440.0,3227.0,2437.0,1574.0,2642.0,1849.0,2586.0,1
23996,166832.0,1,1,1,39,1,-2,-2,-2,-2,...,-867.0,-1298.0,969.0,0.0,0.0,867.0,0.0,32.0,64.0,1
23997,78340.0,1,1,2,30,0,0,0,0,0,...,29343.0,49329.0,54987.0,5007.0,1096.0,1440.0,1935.0,1152.0,1728.0,1
23998,271732.0,2,2,1,41,-1,-1,-1,-1,-1,...,1747.0,4445.0,159.0,0.0,926.0,136.0,3153.0,44.0,599.0,1


In [16]:
evaluation_dict = {'CTGAN new' : 
                   (df_real_credit_card_train, df_real_credit_card_ctgan_test_new),
                    'CTGAN' :
                   (df_real_credit_card_train, df_real_credit_card_ctgan_third),
                   }

qai_columns = ['LIMIT_BAL','SEX','EDUCATION','MARRIAGE','AGE']
risk_column = ['PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6','default.payment.next.month']
builder = EvaluationReportBuilder(evaluation_dict )
# builder.with_privacy(df_real_credit_card_test, qai_columns, risk_column).with_privacy_anonymeter(df_real_credit_card_test).get_evaluation()
# builder.with_privacy(df_real_credit_card_test, qai_columns, risk_column).get_evaluation()
report_ress = builder.with_resemblance().build()
# report.save('report\\report_resemblance_credit_test')

In [17]:
report_ress.display_evaluation()

### ctgan Adult

In [5]:
cat_list_adult = ['workclass','education','marital-status','occupation','relationship','race','sex','native-country','income']

df_real_adult_train = DataLoader('../data/adult_train.csv').get_dataframe(cat_list_adult)
df_real_adult_test = DataLoader('../data/adult_test.csv').get_dataframe(cat_list_adult)

In [6]:
from sdv.metadata import SingleTableMetadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df_real_adult_train)
ctgan = SDVCTGAN_(metadata, df_real_adult_train, verbose=False, cuda=True, epochs=1500)

discrete_columns :  ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']


In [7]:
ctgan.fit(df_real_adult_train)

_fit from child
got it


In [8]:
data = ctgan.sample(32561)

In [15]:
data[data['occupation'] == ' Armed-Forces']

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
316,51,Federal-gov,322582,HS-grad,9,Married-civ-spouse,Armed-Forces,Husband,White,Male,8,0,40,United-States,<=50K
1539,58,Federal-gov,189213,12th,7,Married-civ-spouse,Armed-Forces,Husband,White,Male,14,0,40,United-States,<=50K
2221,49,Federal-gov,239051,Bachelors,13,Married-civ-spouse,Armed-Forces,Husband,White,Male,0,0,40,United-States,<=50K
2392,38,Federal-gov,304390,HS-grad,9,Married-civ-spouse,Armed-Forces,Husband,White,Male,23,0,40,United-States,<=50K
2737,30,Federal-gov,331707,Assoc-acdm,12,Married-civ-spouse,Armed-Forces,Husband,Black,Male,0,1813,40,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29385,32,Federal-gov,298845,Bachelors,13,Married-civ-spouse,Armed-Forces,Husband,White,Male,2,0,40,United-States,<=50K
29710,32,Federal-gov,151602,Some-college,10,Married-civ-spouse,Armed-Forces,Husband,White,Male,14,0,40,United-States,<=50K
29951,61,Federal-gov,46856,Masters,14,Married-civ-spouse,Armed-Forces,Husband,White,Male,0,0,40,United-States,>50K
31160,47,Federal-gov,188701,Some-college,10,Never-married,Armed-Forces,Not-in-family,Black,Male,4060,0,40,United-States,<=50K


In [19]:
data.to_csv('../data/adult_ctgan_test_new.csv', sep=',', index=False)

### ctgan cardio

In [22]:
cat_list_cardio = ['gender','cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio']

df_real_cardio_train = DataLoader('../data/cardio_train.csv').get_dataframe(cat_list_cardio, category_type=str, sep = ',')
df_real_cardio_test = DataLoader('../data/cardio_test.csv').get_dataframe(cat_list_cardio, category_type=str, sep = ',')

In [26]:
from sdv.metadata import SingleTableMetadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df_real_cardio_train)
ctgan = SDVCTGAN_(metadata, df_real_cardio_train, verbose=False, cuda=True, epochs=1500)

discrete_columns :  ['gender', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio']


In [27]:
ctgan.fit(df_real_cardio_train)

_fit from child
got it


In [28]:
data = ctgan.sample(49000)

In [29]:
data.to_csv('../data/cardio_ctgan_test_new.csv', sep=',', index=False)

## Test Pipeline with Cardio - TVAE

In [1]:
# from data_synthesizer.util import  plot_training_loss, ModelType
from data_loader import DataLoader
from data_synthesizer.sdv import SDVCTGAN_, SDVTVAE_
from data_evaluator import histo_plot_utility_compare, ClassifierType, MultivariateEvaluator, UtilityEvaluation, accuracy_compare, UnivariateEvaluator
from data_evaluator.privacy_evaluation import SimilarityType
# from evaluation_report import EvaluationReportBuilder, EvaluationType, EvaluationReport
from data_synthesizer.pipeline import PipelineBuilder, TaskType


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
cat_list_credit_card = ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6', 'default.payment.next.month']
# cat_list_credit_card = ['EDUCATION', 'MARRIAGE', 'PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6', 'default.payment.next.month']
num_list_credit_card = ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4','BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3','PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
cat_list_adult = ['workclass','education','marital-status','occupation','relationship','race','sex','native-country','income']
num_list_adult = ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']
df_real_credit_card_train = DataLoader('../data/credit_card_Train.csv').get_dataframe(cat_list_credit_card, str, drop_identation=True)
df_real_credit_card_test = DataLoader('../data/credit_card_Test.csv').get_dataframe(cat_list_credit_card, str, drop_identation=True)
df_real_credit_card_ctgan_fourth = DataLoader('../data/credit_card_ctgan_training_fourth.csv').get_dataframe(cat_list_credit_card, str)
df_real_adult_train = DataLoader('../data/adult_train.csv').get_dataframe(cat_list_adult, str)
df_real_adult_test = DataLoader('../data/adult_test.csv').get_dataframe(cat_list_adult, str)

### generation pipeline

In [3]:
from sdv.metadata import SingleTableMetadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df_real_adult_train)
ctgan = SDVTVAE_(metadata, df_real_adult_train, epochs= 300, cuda=True)


In [4]:
pipeline_builder = PipelineBuilder(df_real_adult_train, cat_list_adult, num_list_adult, ctgan)
pipeline_builder.add_generation_task()
pipeline_builder.add_fine_tuning_generation_task()
# pipeline_builder.add_sampling_and_reject_task(0.1)
pipeline_builder.add_ressemblance_evaluation_task(df_real_adult_test)
pipeline = pipeline_builder.build()
synth = pipeline_builder.run()

Generation processing:
Epoch 1, Reconstruct Loss:  23.7941,KLD Loss:  0.0878
Epoch 2, Reconstruct Loss:  20.1246,KLD Loss:  0.5573
Epoch 3, Reconstruct Loss:  16.4468,KLD Loss:  1.0638
Epoch 4, Reconstruct Loss:  18.7770,KLD Loss:  1.5021
Epoch 5, Reconstruct Loss:  15.1458,KLD Loss:  2.3038
Epoch 6, Reconstruct Loss:  16.7394,KLD Loss:  2.6371
Epoch 7, Reconstruct Loss:  13.2767,KLD Loss:  3.6936
Epoch 8, Reconstruct Loss:  12.5327,KLD Loss:  4.9614
Epoch 9, Reconstruct Loss:  10.5303,KLD Loss:  4.1699
Epoch 10, Reconstruct Loss:  7.0933,KLD Loss:  4.7018
Epoch 11, Reconstruct Loss:  9.6469,KLD Loss:  5.4988
Epoch 12, Reconstruct Loss:  8.0385,KLD Loss:  5.4767
Epoch 13, Reconstruct Loss:  5.6011,KLD Loss:  7.5573
Epoch 14, Reconstruct Loss:  4.5499,KLD Loss:  6.7569
Epoch 15, Reconstruct Loss:  3.5194,KLD Loss:  7.5395
Epoch 16, Reconstruct Loss:  1.7619,KLD Loss:  6.7587
Epoch 17, Reconstruct Loss:  3.4459,KLD Loss:  8.0093
Epoch 18, Reconstruct Loss: -1.4448,KLD Loss:  7.7408
Epoch

In [5]:
pipeline_builder.results

{'generation_results': {'synthetic_data':        age workclass  fnlwgt      education  education-num  \
  0       20   Private   44905        HS-grad              9   
  1       32   Private  109852        HS-grad              9   
  2       38   Private  124729        HS-grad              9   
  3       23   Private   52677      Bachelors             13   
  4       41   Private  138111        HS-grad              9   
  ...    ...       ...     ...            ...            ...   
  32556   32   Private   27882           10th             10   
  32557   32   Private   27882   Some-college             10   
  32558   32   Private   27882   Some-college             10   
  32559   32   Private   27882   Some-college             10   
  32560   32   Private   27882   Some-college             10   
  
              marital-status          occupation     relationship    race  \
  0            Never-married               Sales        Own-child   White   
  1       Married-civ-spouse   Hand

In [15]:
df_real_adult_train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [12]:
for i, child in enumerate(model._model.encoder.seq.children()):
    if i < 2:
        for param in child.parameters():
            print(param.requires_grad)

True
True


In [5]:
synth

{'generation_results': {'synthetic_data':        age     workclass  fnlwgt      education  education-num  \
  0       44       Private  395942        HS-grad              7   
  1       56     State-gov  207793   Some-college             12   
  2       40       Private   96988        HS-grad             11   
  3       43   Federal-gov   52352   Some-college              5   
  4       39       Private   45930   Some-college             14   
  ...    ...           ...     ...            ...            ...   
  32556   58       Private   26258   Some-college             13   
  32557   27   Federal-gov  172852   Some-college             14   
  32558   31       Private  634264        HS-grad             10   
  32559   72       Private   76253   Some-college              9   
  32560   42       Private   57576            9th             12   
  
              marital-status          occupation    relationship  \
  0            Never-married        Craft-repair         Husband   
  1  

In [6]:
from data_synthesizer.privacy_sampling import get_epsilon


get_epsilon(df_real_adult_train, synth, cat_list_adult, num_list_adult)

0.0499984644206259

In [8]:
train_data_to_evaluate = DataLoader(dataset = df_real_credit_card_train).get_dataframe(cat_list_credit_card)
synth_data_to_evaluate = DataLoader(dataset = synth).get_dataframe(cat_list_credit_card)

uni_evaluators = UnivariateEvaluator(train_data_to_evaluate, synth_data_to_evaluate)
uni_evaluators.evaluate_mode_collapse_values()
mode_collapse = uni_evaluators.get_mode_collapse()

KeyError: 'SEX'

In [7]:
import pandas as pd
mask = pd.Series([False] * len(df_real_credit_card_train))

for feature, values in mode_collapse.items():
    print(values)
    mask |= df_real_credit_card_train[feature].isin(values)

['0', '3', '4', '5', '6']
['0', '3']
['2', '3', '4', '5', '6', '7', '8']
['1', '3', '4', '5', '6', '7', '8']
['1', '3', '4', '5', '6', '7', '8']
['1', '3', '4', '5', '6', '7', '8']
['2', '3', '4', '5', '6', '7', '8']
['2', '3', '4', '5', '6', '7', '8']
['1']


In [8]:
filtered_D = df_real_credit_card_train[mask]
print(filtered_D)

       LIMIT_BAL SEX EDUCATION MARRIAGE  AGE PAY_0 PAY_2 PAY_3 PAY_4 PAY_5  \
2       180000.0   2         5        1   44     0     0    -1    -1    -1   
5        20000.0   1         3        2   32     1     2     0     0     0   
9       360000.0   2         3        1   43    -2    -2    -2    -2    -2   
10      220000.0   2         2        2   38    -1    -1    -1    -1    -1   
12      100000.0   1         1        2   27    -1     2     0     0     0   
...          ...  ..       ...      ...  ...   ...   ...   ...   ...   ...   
23992   280000.0   2         1        1   32     1    -2    -1     0     0   
23993   160000.0   2         2        1   42     0     0     0     0     0   
23996   200000.0   1         1        2   37     2     2     2     2     2   
23998    70000.0   2         2        2   25     0     0     0     0     2   
23999   160000.0   2         2        1   36    -2    -2    -2    -2    -2   

       ... BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2 

In [27]:
filtered_D.reset_index()

,index,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,2,180000.0,2,5,1,44,0,0,-1,-1,...,0.0,6881.0,10340.0,0.0,850.0,0.0,6881.0,10340.0,182.0,0
1,5,20000.0,1,3,2,32,1,2,0,0,...,19588.0,20037.0,19880.0,0.0,1302.0,685.0,748.0,697.0,690.0,0
2,9,360000.0,2,3,1,43,-2,-2,-2,-2,...,15584.0,3195.0,4261.0,805.0,1071.0,15604.0,3195.0,4269.0,3525.0,0
3,10,220000.0,2,2,2,38,-1,-1,-1,-1,...,1621.0,8522.0,4149.0,5575.0,4716.0,1632.0,8559.0,4164.0,10626.0,1
4,12,100000.0,1,1,2,27,-1,2,0,0,...,6746.0,7889.0,0.0,0.0,1000.0,2000.0,3323.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10431,23992,280000.0,2,1,1,32,1,-2,-1,0,...,3508.0,-200.0,-200.0,0.0,3508.0,0.0,0.0,0.0,0.0,1
10432,23993,160000.0,2,2,1,42,0,0,0,0,...,119529.0,122059.0,111378.0,8450.0,5289.0,4022.0,4195.0,3986.0,3958.0,1
10433,23996,200000.0,1,1,2,37,2,2,2,2,...,164182.0,169029.0,172084.0,13500.0,6000.0,0.0,7500.0,6000.0,4000.0,1
10434,23998,70000.0,2,2,2,25,0,0,0,0,...,35122.0,28633.0,28039.0,3000.0,2000.0,4500.0,1200.0,0.0,1200.0,1


In [10]:
ctgan.fit_with_only_rare(filtered_D)
ctgan.reset_sampling()
synth_data_new = ctgan.sample(len(df_real_credit_card_train))

mask_ = pd.Series([False] * len(df_real_credit_card_train))

for feature, values in mode_collapse.items():
    print(values)
    mask_ |= synth_data_new[feature].isin(values)

Epoch 1, Reconstruct Loss:  27.0434,KLD Loss:  4.7940
Epoch 2, Reconstruct Loss:  30.1614,KLD Loss:  5.2331
Epoch 3, Reconstruct Loss:  36.4279,KLD Loss:  5.5586
Epoch 4, Reconstruct Loss:  46.8723,KLD Loss:  4.6960
Epoch 5, Reconstruct Loss:  55.3024,KLD Loss:  2.0771
['0', '3', '4', '5', '6']
['0', '3']
['2', '3', '4', '5', '6', '7', '8']
['1', '3', '4', '5', '6', '7', '8']
['1', '3', '4', '5', '6', '7', '8']
['1', '3', '4', '5', '6', '7', '8']
['2', '3', '4', '5', '6', '7', '8']
['2', '3', '4', '5', '6', '7', '8']
['1']


In [14]:
filtered_D_ = synth_data_new[mask_]
filtered_D_

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,118241.0,2,2,1,35,0,2,2,0,0,...,1914.0,23769.0,17619.0,413.0,586.0,1291.0,2362.0,0.0,3955.0,0
1,160208.0,2,2,1,33,0,2,2,0,0,...,-3570.0,21745.0,32241.0,1296.0,2483.0,455.0,1664.0,306.0,2588.0,0
2,126621.0,2,2,1,35,0,2,2,0,0,...,71.0,21458.0,13619.0,1569.0,1789.0,933.0,1647.0,0.0,246.0,0
3,142494.0,2,2,1,34,0,2,2,0,0,...,-472.0,10138.0,17305.0,2310.0,403.0,0.0,2511.0,412.0,2795.0,0
4,121203.0,2,2,1,31,0,2,2,0,0,...,-20.0,33551.0,10505.0,2247.0,1603.0,857.0,1658.0,0.0,3165.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,132362.0,2,2,1,33,0,2,2,0,0,...,729.0,17296.0,8899.0,0.0,0.0,413.0,2442.0,0.0,3043.0,0
23996,145460.0,2,2,1,38,0,2,2,0,0,...,-1019.0,14361.0,6516.0,138.0,1451.0,609.0,3081.0,340.0,1074.0,0
23997,154833.0,2,2,1,38,0,2,2,0,0,...,3788.0,22496.0,870.0,0.0,1521.0,0.0,589.0,24.0,2726.0,0
23998,152616.0,2,2,1,33,0,2,2,0,0,...,-1257.0,26667.0,6083.0,2482.0,1884.0,668.0,684.0,0.0,2433.0,0


In [28]:
train_data_to_evaluate = DataLoader(dataset = df_real_credit_card_train).get_dataframe(cat_list_credit_card)
synth_data_to_evaluate_new = DataLoader(dataset = df_combined).get_dataframe(cat_list_credit_card)

uni_evaluators = UnivariateEvaluator(train_data_to_evaluate, synth_data_to_evaluate_new)
uni_evaluators.evaluate_mode_collapse_values()
mode_collapse_ = uni_evaluators.get_mode_collapse()

In [29]:
mode_collapse

{'EDUCATION': ['0', '3', '4', '5', '6'],
 'MARRIAGE': ['0', '3'],
 'PAY_0': ['2', '3', '4', '5', '6', '7', '8'],
 'PAY_2': ['1', '3', '4', '5', '6', '7', '8'],
 'PAY_3': ['1', '3', '4', '5', '6', '7', '8'],
 'PAY_4': ['1', '3', '4', '5', '6', '7', '8'],
 'PAY_5': ['2', '3', '4', '5', '6', '7', '8'],
 'PAY_6': ['2', '3', '4', '5', '6', '7', '8'],
 'default.payment.next.month': ['1']}

In [30]:
mode_collapse_

{'EDUCATION': ['0', '3', '4', '5', '6'],
 'MARRIAGE': ['0', '3'],
 'PAY_0': ['2', '3', '4', '5', '6', '7', '8'],
 'PAY_2': ['1', '3', '4', '5', '6', '7', '8'],
 'PAY_3': ['1', '3', '4', '5', '6', '7', '8'],
 'PAY_4': ['1', '3', '4', '5', '6', '7', '8'],
 'PAY_5': ['2', '3', '4', '5', '6', '7', '8'],
 'PAY_6': ['2', '3', '4', '5', '6', '7', '8']}

In [14]:
most_frequent = {}
for column in synth.columns:
    most_common_value = synth[column].value_counts().idxmax()
    most_frequent[column] = most_common_value

In [27]:
df_combined = synth.copy()
most_common_value = most_frequent['EDUCATION']
most_common_value

'2'

In [40]:
(synth_data_new['EDUCATION'] == '0').any()

False

In [15]:

df_combined = synth.copy()
for feature, values in mode_collapse.items():
    print(feature, ' : ', values)
    most_common_value = most_frequent[feature]
    for value in values :
        filter = (synth_data_to_evaluate_new[feature] == value)
        filtered_df = synth_data_to_evaluate_new[filter]
        ten_sampled_df = filtered_df.sample(n=10, random_state=42) if len(filtered_df) >= 10 else filtered_df
        df_combined = pd.concat([df_combined, ten_sampled_df], ignore_index=True)

        filter_most = (df_combined[feature] == most_common_value)
        # filtered_df_most = df_combined[filter]
        random_index = df_combined[filter_most].sample(len(ten_sampled_df)).index
        df_combined = df_combined.drop(random_index)
                


EDUCATION  :  ['0', '3', '4', '5', '6']
MARRIAGE  :  ['0', '3']
PAY_0  :  ['2', '3', '4', '5', '6', '7', '8']
PAY_2  :  ['1', '3', '4', '5', '6', '7', '8']
PAY_3  :  ['1', '3', '4', '5', '6', '7', '8']
PAY_4  :  ['1', '3', '4', '5', '6', '7', '8']
PAY_5  :  ['2', '3', '4', '5', '6', '7', '8']
PAY_6  :  ['2', '3', '4', '5', '6', '7', '8']
default.payment.next.month  :  ['1']


### evaluation pipeline

In [3]:
pipeline_builder = PipelineBuilder(df_real_credit_card_train, cat_list_credit_card, num_list_credit_card)
# pipeline_builder.add_ressemblance_evaluation_task(df_real_credit_card_test, df_real_credit_card_ctgan_fourth )
# pipeline_builder.add_utility_evaluation_task(df_real_credit_card_test, [ClassifierType.XGBOOST, ClassifierType.CART], df_real_credit_card_ctgan_fourth)
qai_columns = ['LIMIT_BAL','SEX','EDUCATION','MARRIAGE','AGE']
risk_column = ['PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6','default.payment.next.month']
pipeline_builder.add_privacy_anonymeter_evaluation_task(df_real_credit_card_test, df_real_credit_card_ctgan_fourth )

pipeline = pipeline_builder.build()
synth = pipeline_builder.run()

Privacy anonymeter evaluation in progress.


Found 21 failed queries out of 500. Check DEBUG messages for more details.
Found 68 failed queries out of 500. Check DEBUG messages for more details.


end


In [14]:
for key in synth['resemblance_evaluation_results'] :
    print(key)

categorical_univariate
numerical_univariate
categorical_multivariate
numerical_multivariate
categorical_numerical_multivariate


In [4]:
synth

{'privacy_anonymeter_evaluation_results': {'singling_univariate': AnonymeterResults(attacks_numbers=500, attacks_succeeded=60, privacy_risk_original=0.12289724937538986, privacy_risk_control=0.1837185152398014, privacy_risk_naive=0.03556819133600645, specific_privacy=0.0),
  'singling_multivariate': AnonymeterResults(attacks_numbers=500, attacks_succeeded=57, privacy_risk_original=0.11694299541815917, privacy_risk_control=0.06867004284042351, privacy_risk_naive=0.04152244529323714, specific_privacy=0.051832277279001394),
  'linkability_attacks': AnonymeterResults(attacks_numbers=2000, attacks_succeeded=2000, privacy_risk_original=0.0343943025567045, privacy_risk_control=0.0249125145058764, privacy_risk_naive=0.003952772498782709, specific_privacy=0.00972403829593118)}}